In [3]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
# import notebook_helper
import sys
import numpy as np
import imodelsx
import imodelsx.process_results
import sasc.data.data
from sasc.data.data import TASKS_D3
import warnings
import scipy.stats
def sem(x):
    '''Compute standard error of the mean, ignoring NaNs
    '''
    with warnings.catch_warnings():
        return scipy.stats.sem(x, ddof=0)
tqdm.pandas()
TASK_NAMES = list(TASKS_D3.keys())
sys.path.append('../experiments/')
# results_dir = '/home/chansingh/mntv1/mprompt/feb18_synthetic_sweep'
results_dir = '/home/chansingh/mntv1/mprompt/feb25'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
r = pd.read_pickle('../results/results_synthetic.pkl')

In [6]:
# simplifying
default_params = {
    'noise_ngram_scores': 0,
    'module_num_restrict': -1,
}
rd = r
for k, v in default_params.items():
    rd = rd[rd[k] == v]

# Top results

In [12]:
with pd.option_context('display.max_colwidth', None, 'display.max_rows', 200):
    d = (
        rd[[
            'task_keyword', 'top_explanation_init_strs',
            'top_contains_keywords',  # 'mean_contains_keywords',
            'top_score_synthetic', 'module_num', 'seed'
        ]]
        .rename(columns={
            'task_keyword': 'GT Explanation',
            'top_explanation_init_strs': 'Generated Explanation',
        })
    )
    d_ = d[d.top_contains_keywords == 0]
    slice_ = pd.IndexSlice[d_.index, d_.columns]

    # s = d.style
    display(
        d
        .style.hide()
        .set_properties(**{'background-color': '#132639'}, subset=slice_)
        .format(precision=2)
    )

GT Explanation,Generated Explanation,top_contains_keywords,top_score_synthetic,module_num,seed
Hillary,hillary clinton,True,0.57,17,2
Hillary,hillary clinton,True,0.57,16,2
Hillary,name hillary or hillary clinton,True,0.53,17,1
Hillary,name hillary or hillary clinton,True,0.53,17,3
evacuation,"concept of evacuation, or the process of leaving a place in an emergency situation",True,0.47,5,1
Hillary,person hillary clinton,True,0.44,16,3
atheistic,belief system of atheism,True,0.42,4,2
abortion,topic of abortion,True,0.40,21,2
abortion,topic of abortion,True,0.40,21,3
abortion,topic of abortion,True,0.40,21,1


# Details

**Selected examples**

In [14]:
pos_examples = [(25, 2), (4, 1), (21, 1), (42, 1), (7, 1),
                (50, 1), (41, 1), (8, 1), (45, 1), (47, 1)]
neg_examples = [(18, 1), (26, 1), (39, 1), (2, 1)]

with pd.option_context('display.max_colwidth', None, 'display.max_rows', 200):
    tab = (
        pd.concat([rd[(rd.module_num == x[0]) & (rd.seed == x[1])]
                  for x in pos_examples + neg_examples])
        [[
            'task_keyword', 'top_explanation_init_strs',
        ]]
        .rename(columns={
                'task_keyword': 'Groundtruth Explanation',
                'top_explanation_init_strs': 'Generated Explanation',
                })
    )
    display(tab)
dummy = [''] * tab.shape[0]
dummy[0] = '\\parbox[c]{1mm}{\\multirow{' + \
    str(len(pos_examples)) + '}{*}{\\rotatebox[origin=c]{90} {Correct}}}'
dummy[len(pos_examples)] = '\\parbox[c]{1mm}{\\multirow{' + str(
    len(neg_examples)) + '}{*}{\\rotatebox[origin=c]{90} {Incorrect}}}'
tab.insert(0, ' ', dummy)
print(tab.style.hide(axis="index").to_latex(hrules=True).replace(
    'science \\\\', 'science \\\\ \n\midrule'))

<>:25: DeprecationWarning: invalid escape sequence '\m'
<>:25: DeprecationWarning: invalid escape sequence '\m'
/tmp/ipykernel_3510213/1985051738.py:25: DeprecationWarning: invalid escape sequence '\m'
  'science \\\\', 'science \\\\ \n\midrule'))


,Groundtruth Explanation,Generated Explanation
312,math,mathematics
347,atheistic,"atheism and related topics, such as theism, religious beliefs, and atheists"
398,abortion,topic of abortion
260,environmentalism,environmentalism and climate action
23,crime,crime and criminal activity
358,sports,sports
114,definition,defining or explaining something
289,shelter,concept of shelter
445,facts,information or knowledge
161,science,science


\begin{tabular}{lll}
\toprule
  & Groundtruth Explanation & Generated Explanation \\
\midrule
\parbox[c]{1mm}{\multirow{10}{*}{\rotatebox[origin=c]{90} {Correct}}} & math & mathematics \\
 & atheistic & atheism and related topics, such as theism, religious beliefs, and atheists \\
 & abortion & topic of abortion \\
 & environmentalism & environmentalism and climate action \\
 & crime & crime and criminal activity \\
 & sports & sports \\
 & definition & defining or explaining something \\
 & shelter & concept of shelter \\
 & facts & information or knowledge \\
 & science & science \\ 
\midrule
\parbox[c]{1mm}{\multirow{4}{*}{\rotatebox[origin=c]{90} {Incorrect}}} & derogatory & negative language and criticism \\
 & ungrammatical & language \\
 & entity & enterprise, estate, and creatures \\
 & subjective & art and expression \\
\bottomrule
\end{tabular}



**More details**

In [16]:
# display full strings
with pd.option_context('display.max_colwidth', None, 'display.max_rows', 200):
    display(
        rd
        .filter(['task_keyword', 'top_explanation_init_strs', 'top_contains_keywords', 'top_score_synthetic', 'module_num'])
        # .filter(['top_score_contains_keywords', 'top_score_synthetic', 'top_explanation_init_strs', 'explanation_init_ngrams', 'module_num'])
        # .filter(['top_explanation_init_strs', 'explanation_init_ngrams', 'top_strs_added'])
        # .filter(['explanation_init_strs', 'explanation_init_ngrams',])
        # .head(n=20)
    )

,task_keyword,top_explanation_init_strs,top_contains_keywords,top_score_synthetic,module_num
367,Hillary,hillary clinton,True,0.570,17
334,Hillary,hillary clinton,True,0.570,16
408,Hillary,name hillary or hillary clinton,True,0.532,17
49,Hillary,name hillary or hillary clinton,True,0.532,17
181,evacuation,"concept of evacuation, or the process of leaving a place in an emergency situation",True,0.471,5
187,Hillary,person hillary clinton,True,0.439,16
410,atheistic,belief system of atheism,True,0.420,4
292,abortion,topic of abortion,True,0.403,21
438,abortion,topic of abortion,True,0.403,21
398,abortion,topic of abortion,True,0.403,21


### Export regexes

In [17]:
from sasc.data.d3 import TASKS_D3

In [29]:
with pd.option_context('display.max_colwidth', None, 'display.max_rows', 200):
    cols = {'index': 'Module name',
            'groundtruth_explanation': 'Dataset explanation', 'check_func': 'Regex check'}
    d = pd.DataFrame(TASKS_D3).T.reset_index()[
        cols.keys()].rename(columns=cols)
    d['Module name'] = d['Module name'].apply(
        lambda x: x.replace('d3_', '').replace('_', '-'))
    print(d.to_latex(index=False).replace('|', '$|$'))

\begin{tabular}{lll}
\toprule
Module name & Dataset explanation & Regex check \\
\midrule
0-irony & contains irony & irony$|$sarcas \\
1-objective & is a more objective description of what happened & objective$|$factual$|$nonpersonal$|$neutral$|$unbias \\
2-subjective & contains subjective opinion & subjective$|$opinion$|$personal$|$bias \\
3-god & believes in god & god$|$religious$|$religion \\
4-atheism & is against religion & atheism$|$atheist$|$anti-religion$|$against religion \\
5-evacuate & involves a need for people to evacuate & evacuat$|$flee$|$escape \\
6-terorrism & describes a situation that involves terrorism & terorrism$|$terror \\
7-crime & involves crime & crime$|$criminal$|$criminality \\
8-shelter & describes a situation where people need shelter & shelter$|$home$|$house \\
9-food & is related to food security & food$|$hunger$|$needs \\
10-infrastructure & is related to infrastructure & infrastructure$|$buildings$|$roads$|$bridges$|$build \\
11-regime change & describ